In [38]:
import pandas as pd
from datetime import datetime, timedelta

project_file = "../data/projects.csv"

# select first 1000 projects
df_projects = pd.read_csv(project_file)[:1000]

# select relevant projects
df_relevant = df_projects[df_projects["relevance"] == "y"]

# check if projects have been updated in the last 90 days
df_relevant["updated_at"] = pd.to_datetime(df_relevant["updated_at"], errors='coerce')

print("Length of df_relevant:", len(df_relevant))

# Convert 'updated_at' to datetime
# The format in the CSV is 'YYYY-MM-DD HH:MM:SS'
df_relevant["updated_at"] = pd.to_datetime(df_relevant["updated_at"], errors='coerce')

# Calculate the cutoff date (3 months ago)
cutoff_date = datetime.now() - timedelta(days=90)

# Filter to only those updated in the last 3 months
df_recent = df_relevant[df_relevant["updated_at"] >= cutoff_date]

df_recent.to_csv("../data/projects_final.csv", index=False)

print("Length of df_recent:", len(df_recent))

with open("projects_final.txt", "w", encoding="utf-8") as dest:
    for index, row in df_recent.iterrows():
        dest.write(f"python3 /tmp/ssimon/config-space/experiments/analysis.py --url={row['html_url']} --name={row['name']}\n")

Length of df_relevant: 768
Length of df_recent: 768


/var/folders/sv/pdgl__6900j8c78ybv1j2hjw0000gp/T/ipykernel_59588/3233421533.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant["updated_at"] = pd.to_datetime(df_relevant["updated_at"], errors='coerce')
/var/folders/sv/pdgl__6900j8c78ybv1j2hjw0000gp/T/ipykernel_59588/3233421533.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant["updated_at"] = pd.to_datetime(df_relevant["updated_at"], errors='coerce')


In [39]:
df = pd.read_csv("../data/projects_final.csv")

# Average size
avg_size = df['size'].mean()
# Largest and smallest size
max_size = df['size'].max()
min_size = df['size'].min()
# Range of languages
languages = df['language'].dropna().unique()
# Average stars
avg_stars = df['stargazers_count'].mean()

most_common_language = df['language'].mode()[0]
language_counts = df['language'].value_counts()
print(f"Most common language: {most_common_language} ({language_counts[most_common_language]} repositories)")

print(f"Average repository size: {avg_size/1024:.2f} MB")
print(f"Largest repository size: {max_size/1024/1024:.2f} GB")
print(f"Smallest repository size: {min_size} kB")
print(f"Number of unique languages: {len(languages)}")
print(f"Languages: {languages}")
print(f"Average number of stars: {avg_stars:.2f}")

python_count = language_counts['Python']
total_count = len(df)
python_percent = (python_count / total_count) * 100
print(f"Percentage of repositories written in Python: {python_percent:.2f}%")

largest_project = df.loc[df['size'].idxmax()]
print(f"Largest project: {largest_project['name']} ({largest_project['full_name']})")
print(f"Size: {largest_project['size']/1024/1024:.2f} GB")
print(f"URL: {largest_project['html_url']}")

Most common language: Python (151 repositories)
Average repository size: 323.68 MB
Largest repository size: 11.50 GB
Smallest repository size: 41 kB
Number of unique languages: 40
Languages: ['Markdown' 'JavaScript' 'TypeScript' 'C' 'C++' 'HTML' 'Shell' 'Python'
 'MDX' 'Dart' 'Jupyter Notebook' 'Go' 'Batchfile' 'C#' 'Java' 'Rust'
 'Vim Script' 'Vue' 'CSS' 'Blade' 'Clojure' 'Zig' 'Assembly' 'Ruby'
 'Kotlin' 'Julia' 'Astro' 'PHP' 'Roff' 'Swift' 'Scala' 'Lua' 'Haskell'
 'Svelte' 'V' 'PowerShell' 'LLVM' 'TeX' 'Jinja' 'Elixir']
Average number of stars: 49068.56
Percentage of repositories written in Python: 19.66%
Largest project: backstage (backstage/backstage)
Size: 11.50 GB
URL: https://github.com/backstage/backstage


In [40]:
# Show all projects with size smaller than 1000 kB
small_projects = df[df['size'] < 1000][['name', 'full_name', 'size', 'html_url']]
print(small_projects["html_url"])


198             https://github.com/necolas/normalize.css
317                  https://github.com/karpathy/nanoGPT
635       https://github.com/VincentGarreau/particles.js
682                 https://github.com/meta-llama/llama3
701    https://github.com/feder-cr/Jobs_Applier_AI_Ag...
704    https://github.com/jamiebuilds/the-super-tiny-...
Name: html_url, dtype: object
